# Documentation 
**Disclaimer**: *this document is not meant to be neither a formal nor an exhaustive description of the iterative method to solve the eigenvalue problem. The aim of this text is only to provide the necessary and minimal information needed for the reader to understand the code implementation. We will use these pages to document, explain and justify the choice made from a numerical and scientific computing standpoint*.

## Problem statement
Given a matrix $\boldsymbol{A} \in \mathbb{C}^{n, n}$, with $n \in \mathbb{N}$ the matrix dimension, the eigenvalue problem can be formulated as finding the eigenpair $\{(\lambda_i, \boldsymbol{v}_i)\}_{i=1} ^ n$, with $\lambda_i \in \mathbb{C}$, $\boldsymbol{v}_i \in \mathbb{C}^{n ,1}$ and $\boldsymbol{v}_i \ne \boldsymbol{0}$ such that 
$$\boldsymbol{A} \boldsymbol{v}_i = \lambda_i \boldsymbol{v}_i, \quad i=1, 2, ..., n $$

$\lambda_i$ are called eigenvalue, while $\boldsymbol{v}_i$ are the corresponding eigenvectors. Theoretically, finding the matrix's eigenvalue is possible by imposing the following condition:
$$ \det(A-\lambda I)=0 $$
which lead to the well known characteristic polynomial of degree $n$. The eigenvalue are the roots of the characteristic polynomial. Although correct, this apporach is not viable, due to the difficulties in both expressing the characteristic polynomial and find its roots, when the Matrix gets very large.

Numerical methods takle the eigenproblem using a different strategy, most of the time being a iterative methods. Rather than aspire to get the exact solution, they seek after an approximation of the solution, which hopefully, under a set of conditions, converges to the exact solution.
Among all the method devolped to solve the eigenproblem, the power method, along with its variants suct that the inverse power method and power method with shift, and the QR are the widest spread and the most used.

## Power method
Let $\boldsymbol{A}$ being a matric and $\lambda _i$ the eigenvalue already sorted accordingly to their module. If
$$ |\lambda_1| > |\lambda _i | \quad i=2, 3, ..., n$$
then the power method allow to recover the eigenpair $(\lambda_1, \boldsymbol{v}_1)$ by applying iteratively, the following steps


$$
\begin{array}{l}
\textbf{Power Method Algorithm} \\
\textbf{Input:} A \in \mathbb{C}^{n \times n}, \text{initial vector } x_0, \text{ tolerance } \text{tol}, \text{ max iterations } \text{max\_iter} \\
\textbf{Output:} \lambda \text{ (dominant eigenvalue approximation), } x \text{ (eigenvector approximation)} \\

1.\ \text{Initialize } x = x_0 \text{ (random or chosen guess)} \\
2.\ \text{Normalize } x \text{: } y^{(1)} = x/ \| x\| \\
3.\ \lambda_{\text{old}} = 0 \\

4.\ \textbf{for } k = 1 \textbf{ to } \text{max\_iter} \textbf{ do} \\
\quad 5.\ x^{(k+1)} = A \cdot y^{(k)} \\
\quad 6.\  y^{(k+1)} = x^{(k+1)}/ \| x^{(k+1)}\|  \\
\quad 7.\  \lambda^{(k+1)} =  (y^{(k+1)})^H \boldsymbol{A} y^{(k+1)} \\
\quad 8.\ \textbf{if } |\lambda^{(k+1)} - \lambda^{(k)}| < \text{tol} \textbf{ then} \\
\quad \quad \text{Break (convergence reached)} \\

9.\ \textbf{Return } \lambda, x
\end{array}
$$

By exploiting Matrix properties, it is also possible to find the eigenvalue with the smallest norm, and its associated eigenvector, or either the the eigenvalue closest to a given number.

## QR algorithm
Being the problem we are interested in to solve incolve a symmetric Matrix, we will start introduce the QR alforithm in a general way and specialize it for the case of symmetric matrices as soon as we have the chance.

The QR algorithm can be divide in to two stages: during the first stage, the matrix is reduced, by means of similar trasformation, into a Hessemberg matrix, or, for Hermetian matrix, into a tridiagonal matrix. Although the QR algorithm can be applied directly to a full matrix, it converges faster if it is applied to Hessemberg Matrix or triagular matrices, with also a smaller computational cost.

## Profiling

In this section we show the profiling and function's optimization based on the profiling analysys. 

### About the algorithm
Given the symmetry of the matrix $\boldsymbol{A}$, we choose the algorithm specialized for this kind of matrices, that allowed for the lowest computational complexity. For instance, to ease the convergence of the QR algrithm, a similarity reduction of the original matrices to a "simpler" matrix is required. If for a general matrices, the reduction using the Householder reflector leads to a quasi upper triangular Hessemberg matrix. The cost of this reduction amounts to $O(\frac{14}{3} n^3)$.
If the original matrix is symmetric, the Hessemberg matrix must be symmetric too, leading to a tridiagonal Matrix. The algorithm implied for this reduction is the Lanczos algorithm, which scales approximatly as $O(2n^2)$. Apart from this algorithm optimization the Lanczos algorithm does not lend itsel to any kind of optimization. Whenever possible, to improve performance, numpy function and operator are used to improve performance.



In [2]:
import numpy as np
from numba import jit, prange
from line_profiler import profile, LineProfiler
from time import time

def Lanczos_PRO_original(A, q, m=None, toll=np.sqrt(np.finfo(float).eps)):
    """
    Perform the Lanczos algorithm for symmetric matrices.
    This function computes an orthogonal matrix Q and tridiagonal matrix T such that A ≈ Q * T * Q.T,
    where A is a symmetric matrix. The algorithm is useful for finding a few eigenvalues and eigenvectors
    of large symmetric matrices.
    Args:
        A (np.ndarray): A symmetric square matrix of size n x n.
        q (np.ndarray): Initial vector of size n.
        m (int, optional): Number of eigenvalues to compute. Must be less than or equal to n.
                           If None, defaults to the size of A.


        tuple: A tuple (Q, alpha, beta) where:


            - Q (np.ndarray): Orthogonal matrix of size n x m.


            - alpha (np.ndarray): Vector of size m containing the diagonal elements of the tridiagonal matrix.


            - beta (np.ndarray): Vector of size m-1 containing the off-diagonal elements of the tridiagonal matrix.
    Raises:
        ValueError: If the input matrix A is not square or if m is greater than the size of A.
    """
    if m == None:
        m = A.shape[0]

    if A.shape[0] != A.shape[1]:
        raise ValueError("Input matrix A must be square.")

    if A.shape[0] != q.shape[0]:
        raise ValueError("Input vector q must have the same size as the matrix A.")
    q = q / np.linalg.norm(q)
    Q = np.array([q])
    r = A @ q
    alpha = []
    beta = []
    alpha.append(q @ r)
    r = r - alpha[0] * q
    beta.append(np.linalg.norm(r))
    count = 0
    for j in range(1, m):
        q = r / beta[j - 1]
        for q_basis in Q[:-1]:
            if np.abs(q @ q_basis) > toll:
                for q_bbasis in Q[:-1]:
                    q = q - (q @ q_bbasis) * q_bbasis
                    count += 1
                break
        q = q / np.linalg.norm(q)
        Q = np.vstack((Q, q))
        r = A @ q - beta[j - 1] * Q[j - 1]
        alpha.append(q @ r)
        r = r - alpha[j] * q
        beta.append(np.linalg.norm(r))

        if np.abs(beta[j]) < 1e-15:
            return Q, alpha, beta[:-1]


    return Q, alpha, beta[:-1]


#@jit(parallel=True)  
def Lanczos_PRO(A, q,  m=None, toll=np.sqrt(np.finfo(float).eps)):
    """
    Lanczos algorithm for symmetric matrices
    :param A: symmetric matrix square matrix of size n
    :param q: initial vector
    :param m: number of eigenvalues to compute. m<=n
    :param toll: tolerance for the computation

    :return: Q, alpha, beta
    Q: orthogonal matrix of size n x m
    alpha: vector of size m. Diagonal of the tridiagonal matrix
    beta: vector of size m-1. Upper diagonal of the tridiagonal matrix
    """
    if m==None:
        m=A.shape[0]
        
    q=q/np.linalg.norm(q)
    #Q=np.array([q])
    Q = np.zeros((m, A.shape[0]))
    Q[0] = q
    r=A@q
    alpha=[]
    beta=[]
    alpha.append(q@r)
    r=r-alpha[0]*q
    beta.append(np.linalg.norm(r))

    for j in range(1, m):
        q=r/beta[j-1]
        if np.any(np.abs(q@Q[:j-1].T)>toll):
            for q_bbasis in Q[:j-1]:
                q = q - (q @ q_bbasis) * q_bbasis
            # for i in prange(j-1):
            #     q = q - (q @ Q[i]) * Q[i]

        q=q/np.linalg.norm(q)
        Q[j]=q
        r=A@q-beta[j-1]*Q[j-1]
        alpha.append(q@r)
        r=r-alpha[j]*q
        beta.append(np.linalg.norm(r))

        if np.abs(beta[j])<1e-15:
            
            return Q, alpha, beta[:-1]
    return Q, alpha, beta[:-1]



In [3]:
np.random.seed(0)
size = 1000
A = np.random.rand(size, size)
A = (A + A.T) / 2
A=np.array(A, dtype=float)
q = np.random.rand(size)
lp = LineProfiler()
lp.add_function(Lanczos_PRO_original)
result = lp.run('Lanczos_PRO_original(A, q)')
#lp.print_stats()
#Q, alpha, beta = Lanczos_PRO(A, q)



t_s=time()
lp.add_function(Lanczos_PRO)
result = lp.run('Lanczos_PRO(A, q)')
lp.print_stats()

t_e=time()
print(f"Optimized function time: {t_e-t_s}")



Timer unit: 1e-09 s

Total time: 4.87741 s
File: /tmp/ipykernel_27411/3254536551.py
Function: Lanczos_PRO_original at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def Lanczos_PRO_original(A, q, m=None, toll=np.sqrt(np.finfo(float).eps)):
     7                                               """
     8                                               Perform the Lanczos algorithm for symmetric matrices.
     9                                               This function computes an orthogonal matrix Q and tridiagonal matrix T such that A ≈ Q * T * Q.T,
    10                                               where A is a symmetric matrix. The algorithm is useful for finding a few eigenvalues and eigenvectors
    11                                               of large symmetric matrices.
    12                                               Args:
    13                                                   A (np.ndarray): A s

From the line profiler we can see that the most expensive operation is associated with reorthogonalization. To improve this a slight modified version was implemented. Also the np.vstack operation was quite costly, so it was replaced with a simple assignment. This little adjustment allowed to almost halve the cost of the Lanczos Algorithm. In the final version the decorator jit, from numba packeage was additionally used. The only portion of code that could be paralleliza using prange is the for associated with reorthogonalization, but this slows down the performance.

### QR algorithm

The QR algorithm, computed with the Given rotation is difficult to be parallelized. By exploiting the tridiagonal shape fo the matrix, it can be optimized.

In [4]:
def QR_method(A_copy, tol=1e-10, max_iter=100):
    A = A_copy.copy()
    iter = 0
    Q = np.eye(A.shape[0])
    
    # Correctly preallocate as a 2D array (n-1, 2)
    Matrix_trigonometry = np.zeros((A.shape[0] - 1, 2))

    while np.linalg.norm(np.diag(A, -1), np.inf) > tol and iter < max_iter:
        # Compute Givens rotation
        for i in range(A.shape[0] - 1):
            c = A[i, i] / np.sqrt(A[i, i] ** 2 + A[i + 1, i] ** 2)
            s = -A[i + 1, i] / np.sqrt(A[i, i] ** 2 + A[i + 1, i] ** 2)
            Matrix_trigonometry[i, :] = [c, s]  
            R=np.eye(A.shape[0])
            # Apply the Givens rotation to A (modify in place)
            R[i:i+2, i:i+2] = np.array([[c, -s], [s, c]])
            A= R @ A
            A[i+1, i] = 0 


        Q=np.eye(A.shape[0])
        i=0
        Q[0:2, 0:2]=np.array([[Matrix_trigonometry[i, 0], Matrix_trigonometry[i, 1]], [-Matrix_trigonometry[i, 1], Matrix_trigonometry[i, 0]]])
        for i in range(1, A.shape[0]-1):
            R=np.eye(A.shape[0])
            R[i: i+2, i:i+2]=np.array([[Matrix_trigonometry[i, 0], Matrix_trigonometry[i, 1]], [-Matrix_trigonometry[i, 1], Matrix_trigonometry[i, 0]]])
            Q = Q@R
        A=A@Q
        A = A @ Q  # Update A
        iter += 1

    if iter == max_iter:
        print("QR method did not converge")

    return np.diag(A), Q

#@jit(nopython=True)
def QR_method_optimized(A_copy, tol=1e-10, max_iter=100):
    A = A_copy.copy()
    iter = 0
    Q = np.eye(A.shape[0])
    
    # Correctly preallocate as a 2D array (n-1, 2)
    Matrix_trigonometry = np.zeros((A.shape[0] - 1, 2))
    d=np.zeros(A.shape[0])
    #while np.linalg.norm((np.diag(A, -1)), np.inf) > tol and iter < max_iter:
    while np.linalg.norm((np.diag(A, 0)-d)/np.diag(A, 0), np.inf) > tol and iter < max_iter:
        # Compute Givens rotation
        d=np.diag(A, 0)
        for i in range(A.shape[0] - 1):
            c = A[i, i] / np.sqrt(A[i, i] ** 2 + A[i + 1, i] ** 2)
            s = -A[i + 1, i] / np.sqrt(A[i, i] ** 2 + A[i + 1, i] ** 2)
            Matrix_trigonometry[i, :] = [c, s]  

            # Apply the Givens rotation to A (modify in place)
            R = np.array([[c, -s], [s, c]])
            A[i:i+2, i:] = R @ A[i:i+2, i:]
            A[i+1, i] = 0 

        # Construct full Q matrix from stored Givens rotations
        Q = np.eye(A.shape[0])
        for i in range(A.shape[0] - 1):
            R=np.array([[Matrix_trigonometry[i, 0], Matrix_trigonometry[i, 1]], [-Matrix_trigonometry[i, 1], Matrix_trigonometry[i, 0]]])
            Q[:, i:i+2] = Q[:, i:i+2]@R
        A = A @ Q  # Update A
        iter += 1

    return np.diag(A), Q





np.random.seed(0)
size = 300
A = np.random.rand(size, size)
A = (A + A.T) / 2
x0 = np.random.rand(size)

q, alpha, beta = Lanczos_PRO(A, x0, size)

T=np.diag(alpha)+np.diag(beta, k=1)+np.diag(beta, k=-1)
QR_method_optimized(T)
lp_QR = LineProfiler()
lp_QR.add_function(QR_method)
result = lp_QR.run('QR_method(T)')


lp_QR.add_function(QR_method_optimized)
result = lp_QR.run('QR_method_optimized(T)')
lp_QR.print_stats()
# t_s=time()
# result= QR_method_optimized(T)
# t_e=time()
# print(f"Optimized function time: {t_e-t_s}")



KeyboardInterrupt: 

The first simple optimization concern the memory. The Rotation matrix at each iteration is completly defined by the value of the sine and cosine (s, c) of the rotation angle, which can be assembled in a $2 \times 2 $ matrix, and the position of this block in the $n \times n $ rotation matrix $G(i, \theta_i)$ matrix, with $i=0, 1, 2, ..., n-2$. Therefore rather than storing $n-1$ $n \times n$ rotation matrix, it is possible store all the inforamtion they carry out in a $(n-1)\times 2$ matrix in which the $i$-th row has the value of sine and cosine associated with the matrix $G(i, \theta_i)$.

Profiling the QR method shows as the most expensive operatation are the matrix matrix multiplication, which adds up to the 92% of the function total run. Knowing the structure of the matrix $G(i, \theta_i)$ allow to reduce the size of the matrix multiplication (see lines 53 and 60). A final 98% speed up of the code was achieved.


In [ ]:
np.random.seed(0)
size = 5000
A = np.random.rand(size, size)
A = (A + A.T) / 2
x0 = np.random.rand(size)
q, alpha, beta = Lanczos_PRO(A, x0, size)

t_s=time()
np.linalg.eig(A)
t_e=time()

print(f"{t_e-t_s: .4e}")

# T=np.diag(alpha)+np.diag(beta, k=1)+np.diag(beta, k=-1)
# t_s=time()
# result= QR_method_optimized(T, max_iter=5000, tol=1e-6)
# t_e=time()
# print(f"Optimized function time: {t_e-t_s}")



 8.5068e+01
